In [1]:
import numpy as np
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.4.0-dev20200712
Eager mode:  True
Hub version:  0.8.0
GPU is available


In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [4]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=40,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 40ms/step - loss: 0.1518 - accuracy: 0.9444 - val_loss: 0.3087 - val_accuracy: 0.8673
Epoch 2/40
30/30 [==============================] - 1s 42ms/step - loss: 0.1431 - accuracy: 0.9493 - val_loss: 0.3097 - val_accuracy: 0.8681
Epoch 3/40
30/30 [==============================] - 1s 41ms/step - loss: 0.1347 - accuracy: 0.9533 - val_loss: 0.3117 - val_accuracy: 0.8681
Epoch 4/40
30/30 [==============================] - 1s 42ms/step - loss: 0.1262 - accuracy: 0.9565 - val_loss: 0.3139 - val_accuracy: 0.8685
Epoch 5/40
30/30 [==============================] - 1s 42ms/step - loss: 0.1182 - accuracy: 0.9599 - val_loss: 0.3165 - val_accuracy: 0.8690
Epoch 6/40
30/30 [==============================] - 1s 42ms/step - loss: 0.1110 - accuracy: 0.9633 - val_loss: 0.3203 - val_accuracy: 0.8690
Epoch 7/40
30/30 [==============================] - 1s 42ms/step - loss: 0.1040 - accuracy: 0.9667 - val_loss: 0.3280 - val_accuracy: 0.8726
Epoch 8/40
30

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.6296 - accuracy: 0.8428
loss: 0.630
accuracy: 0.843
